# On modelling optimization problems via Julia JuMP

## Prof. Mayron César O. Moreira 

**Universidade Federal de Lavras (UFLA)**  
**Department of Computer Science**  
**Lavras, Minas Gerais, Brazil**  

*Università degli Studi di Modena e Reggio Emilia (UNIMORE)*  
*Reggio Emilia, Italy*

## Parameters settings

Parametrization is an essential phase in practical and research problems. As follows, we will present some of the most used parameters in the optimization models. We focus our examples for Cbc solver. For further information about popular solvers such as **Gurobi** and **CPLEX**, see https://github.com/JuliaOpt/Gurobi.jl and https://github.com/JuliaOpt/CPLEX.jl, respectively.

In this example, we parametrize:

* *seconds*: time limit (it must be a Float64);
* *logLevel*: equal to one if we want to enable solution output;
* *maxSolutions*: maximum number of feasible solutions allowed during the search;
* *maxNodes*: maximum number of branch \& bound nodes allowed during the search;
* *allowableGap*: optimility gap allowed to stop searching;
* *threads*: number of threads for parallel branch \& bound.

In [7]:
# Suppose a random instance of knapsack problem
n = 1000 # Number of items
c = 400 # Capacity
w = [rand(50:150) for i=1:n] # Weight
v = [rand(50:100) for i=1:n] # Profit

# Importing OR-lbraries
using JuMP, Cbc

#= Our model will have:
    seconds = 600
    logLevel = 1 (we will output log information)
    maxSolutions = 1000
    maxNodes = 1000
    allowableGap = 10^-3
    threads = 1 (single thread)
=#
model = Model(with_optimizer(Cbc.Optimizer, seconds=600, logLevel=1, 
        maxSolutions=1000,maxNodes=1000,allowableGap=1e-3,threads=1))

# x[i] \in \{0,1\}: one if i is in the knapsack
@variable(model, x[1:n], Bin)

# Maximize profit
@objective(model, Max, sum(v[i]*x[i] for i=1:n))

# Capacity constraints
@constraint(model, sum(w[i]*x[i] for i=1:n) <= c)

# Let´s solve it
optimize!(model)

println("Solution: ", [i for i=1:n if value(x[i]) >= 0.9])

Welcome to the CBC MILP Solver 
Version: 2.9.9 
Build Date: Dec 31 2018 

command line - Cbc_C_Interface -threads 1 -maxSolutions 1000 -seconds 600 -allowableGap 0.001 -maxNodes 1000 -logLevel 1 -solve -quit (default strategy 1)
threads was changed from 0 to 1
seconds was changed from 1e+100 to 600
allowableGap was changed from 1e-10 to 0.001
maxNodes was changed from 2147483647 to 1000
Continuous objective value is 745.778 - 0.00 seconds
Cgl0004I processed model has 1 rows, 958 columns (958 integer (919 of which binary)) and 958 elements
Cutoff increment increased from 1e-05 to 0.9999
Cbc0038I Initial state - 1 integers unsatisfied sum - 0.314815
Cbc0038I Solution found of -745.778
Cbc0038I Branch and bound needed to clear up 1 general integers
Cbc0038I Full problem 1 rows 958 columns, reduced to 1 rows 14 columns
Cbc0038I Cleaned solution of -680
Cbc0038I Before mini branch and bound, 957 integers at bound fixed and 0 continuous
Cbc0038I Full problem 1 rows 958 columns, reduced to 0 

Now suppose that you want to start your model with a feasible solution obtained by a heuristic, for example. This warmstart follows this pattern:

In [8]:
model = Model(with_optimizer(Cbc.Optimizer))

@variable(model, y[1:10] >= 0, Int)

init_var = [2, 3, 1, 9, 2, 1, 2, 1, 2, 8]

for i=1:10 
    set_start_value(y[i], init_var[i])
end